In [ ]:
import openai
import os
from dotenv import load_dotenv

load_dotenv('.env')


openai.api_key = os.getenv('OPENAI_API_KEY')




In [ ]:
class ChatBot():
    
    def answer(self, user_content):
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a chatbot. Hold a conversation. Respond using markdown."},
                {"role": "user", "content": user_content},
            ],
        )
        return response["choices"][0]["message"]["content"]

In [ ]:
mes = 'what is your name?'
chat = ChatBot()
a = chat.answer(mes)


In [ ]:
a

In [ ]:
type(a)

In [ ]:
from chatbot import ChatBot 

# Instantiate the chatbot and layout
chatbot = ChatBot()

In [ ]:
def parse_sources(sources):
    # Create an empty list to store results
    parsed_sources = []
    # Iterate over all sources
    for source in sources:
        # Get the metadata from the source
        metadata = source['metadata']
        # Get the source name, or use a default if not found
        source_name = metadata.get('source', 'Unknown Source')
        # Get the page number, or use a default if not found
        page_number = metadata.get('page', 'Unknown Page')
        # Create the string and add it to the results
        parsed_sources.append(f"Source: {source_name}, Page: {page_number}")
    # Return the results
    return parsed_sources


In [ ]:
def parse_page_content(page_content):
    lines = page_content.split('\n')
    parsed_content = "\n".join(line for line in lines if line.strip())  # remove empty lines
    return parsed_content


In [ ]:
chat = chatbot.chat_bot_response('what is the law on gardeing?', [])

In [ ]:
pc = chat['source_documents'][0].page_content

In [ ]:
parse_page_content(pc)

In [ ]:
query = "Does pruning of a native tree trigger a permit requirement under the Significant landscape overlay schedule 2"
chat = chatbot.chat_bot_response(query, [])

In [6]:
chat

{'question': 'Does pruning of a native tree trigger a permit requirement under the Significant landscape overlay schedule 2',
 'chat_history': [],
 'answer': 'Yes, pruning of a native tree does trigger a permit requirement under the Significant Landscape Overlay Schedule 2. This is stated in clause 42.03-2 of the Nillumbik Planning Scheme, which requires a permit to be obtained for the construction of a building or to carry out works, including the pruning or lopping of the trunk of a native tree, unless a schedule to the overlay specifically states that a permit is not required. Therefore, if the schedule to the overlay does not specifically state that a permit is not required for pruning of a native tree, then a permit must be obtained.\n\nThe relevant clause is:\n\nVC224 Permit requirement\nA permit is required to:\n- Construct a building or to carry out works. This does not apply:\n- If a schedule to this overlay specifically states that a permit is not required.\n- To the conduct 

In [ ]:
query = "What are the laws relating to development of outdoor areas"
chat = chatbot.chat_bot_response(query, [])

In [ ]:
chat


In [ ]:
# load the documents from the docs directory
loader = PyPDFDirectoryLoader("docs/")
docs = loader.load()

#split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

In [ ]:
url = "<---qdrant cloud cluster url here --->"
api_key = "<---api key here--->"
qdrant = Qdrant.from_documents(
    docs, embeddings, 
    url, prefer_grpc=True, api_key=api_key, 
    collection_name="my_documents",
)